In [1]:
using InteractiveUtils, Pkg, Test

Base.banner()

Pkg.activate(@__DIR__)
Pkg.status()

using JET

               _
   _       _ _(_)_     |  Documentation: https://docs.julialang.org
  (_)     | (_) (_)    |
   _ _   _| |_  __ _   |  Type "?" for help, "]?" for Pkg help.
  | | | | | | |/ _` |  |
  | | |_| | | | (_| |  |  Version 1.9.1 (2023-06-07)
 _/ |\__'_|_|_|\__'_|  |  Official https://julialang.org/ release
|__/                   |



  Activating project at `~/julia/JET-20230703`


Status `~/julia/JET-20230703/Project.toml`
  [c1f8dc73] Demo1 v0.1.0 `Demo1`
  [c4e90434] Demo2 v0.1.0 `Demo2`
  [c3a54625] JET v0.8.6
  [bc7fa6ce] JuliaHub v0.1.2 `JuliaHub`


# Basic Usage

- Interactive entry point: `[@]report_call`, `[@]report_opt`
  - `report_call` for detecting bugs
  - `report_opt` for detecting performance problems
- Analyze individual call-graphs (works like `[@]code_typed`)

In [2]:
report_call(sum, (Vector{Int},))

No errors detected


In [3]:
@report_call sum("julia")

═════ 2 possible errors found ═════
┌ sum(a::String) @ Base ./reduce.jl:559
│┌ sum(a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:559
││┌ sum(f::typeof(identity), a::String) @ Base ./reduce.jl:530
│││┌ sum(f::typeof(identity), a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:530
││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:302
│││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:302
││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:170
│││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String; init::Base._InitialValue) @ Base ./reduce.jl:170
││││││││┌ mapfoldl_impl(f::typeof(identity), op::typeof(Base.add_sum), nt::Base._InitialValue, itr::String) @ Base ./reduce.jl:44

# Notable features for package development

- Test.jl integration
- `report_package`
- The typo detection mode
- Report filters

# Test.jl integration

JET provides smooth integration with Test.jl:
- Allows users to use JET in their test suite
  - `[@]report_call`, `[@]report_opt` <-> `[@]test_call`, `[@]test_opt`
  - `report_package`, `report_file` <-> `test_package`, `test_file`
- Results are included in a test set summary
- Also supports Test.jl features like `broken` and `skip`

In [4]:
function test_func(a)
    return a + 1
end

@test_call test_func("julia")

JET-test failed at /Users/aviatesk/julia/JET-20230703/20230703.ipynb:5
  Expression: #= /Users/aviatesk/julia/JET-20230703/20230703.ipynb:5 =#

 JET.@test_call test_func("julia")
  ═════ 1 possible error found ═════
  ┌ test_func(a::String) @ Main /Users/aviatesk/julia/JET-20230703/20230703.ipynb:2
  │ no matching method found `+(::String, ::Int64)`: (a::String + 1)
  └────────────────────
  


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [5]:
@testset "JET code quality test" begin
    @test_call test_func("julia")
    @test_call test_func(1)
end

JET code quality test: JET-test failed at /Users/aviatesk/julia/JET-20230703/20230703.ipynb:2
  Expression: #= /Users/aviatesk/julia/JET-20230703/20230703.ipynb:2 =# JET.@test_call test_func("julia")
  ═════ 1 possible error found ═════
  ┌ test_func(a::String) @ Main /Users/aviatesk/julia/JET-20230703/20230703.ipynb:2
  │ no matching method found `+(::String, ::Int64)`: (a::String + 1)
  └────────────────────
  
Test Summary:         | Pass  Fail  Total  Time
JET code quality test |    1     1      2  0.1s


TestSetException: Some tests did not pass: 1 passed, 1 failed, 0 errored, 0 broken.

In [6]:
@testset "JET code quality test" begin
    @test_call skip=true test_func("julia")
    @test_call test_func(1)
end

Test Summary:         | Pass  Broken  Total  Time
JET code quality test |    1       1      2  0.0s


Test.DefaultTestSet("JET code quality test", Any[Test Broken
  Skipped: #= /Users/aviatesk/julia/JET-20230703/20230703.ipynb:2 =# JET.@test_call test_func("julia")], 1, false, false, true, 1.688387654056474e9, 1.688387654056574e9, false)

Effective when:
- your package is simple, and type-stable
- most package code is reachable from entry point method

Packages using JET in their CI in this way: 
- JET: <https://github.com/aviatesk/JET.jl/blob/master/test/self_check.jl>
- CliMA ecosystem

# `report_package`

`report_package` is an easy entry point to analyze package code.

- Interactive entry points (`report_call`)
  - Bad: Require programmers to give concrete input types
  - Good: More accurate
- `report_package`: Use method signatures as analysis entry point
  - Good: Easy to use
  - Bad: Less accurate

Ideally package test suite should use the interactive entry points if possible.
`report_package` is still useful for a first inspection, etc.

## Improved affinity with Pkg.jl system

No need to instantiate/activate a target package environment beforehand.
Just install it, and give its name to `report_package`.

In [7]:
report_package("Demo1")

[toplevel-info] virtualized the context of Main (took 0.009 sec)
[toplevel-info] entered into /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl


[toplevel-info]  exited from /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl (took 0.235 sec)
[toplevel-info] analyzing from top-level definitions (1/3)

[toplevel-info] analyzing from top-level definitions (3/3)
[toplevel-info] analyzed 3 top-level definitions (took 0.45 sec)


═════ 2 possible errors found ═════
┌ callsum2(a::String) @ Demo1 /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl:7
│┌ sum(a::String) @ Base ./reduce.jl:559
││┌ sum(a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:559
│││┌ sum(f::typeof(identity), a::String) @ Base ./reduce.jl:530
││││┌ sum(f::typeof(identity), a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:530
│││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:302
││││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:302
│││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:170
││││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String; init::Base._InitialValue) @ Base ./reduce.jl:170
│││││││││┌ mapfoldl_impl(f::typeof(

## `target_defined_modules` configuration:

- Only report problems that are detected within the module context of a target package.
- Allows you to focus on your code.

In [8]:
# TODO-1 use `target_defined_modules=true` and filter out errors from `Core.Compiler`
# TODO-2 put the configuration into a configuration file

report_package("Demo2")

[toplevel-info] virtualized the context of Main (took 0.001 sec)
[toplevel-info] entered into /Users/aviatesk/julia/JET-20230703/Demo2/src/Demo2.jl
[toplevel-info]  exited from /Users/aviatesk/julia/JET-20230703/Demo2/src/Demo2.jl (took 0.021 sec)
[toplevel-info] analyzing from top-level definitions (1/11)

[toplevel-info] analyzing from top-level definitions (3/11)

[toplevel-info] analyzing from top-level definitions (9/11)

[toplevel-info] analyzing from top-level definitions (11/11)
[toplevel-info] analyzed 11 top-level definitions (took 1.888 sec)


═════ 23 possible errors found ═════
┌ infer(tt::Any) @ Demo2 /Users/aviatesk/julia/JET-20230703/Demo2/src/Demo2.jl:10
│┌ infer(tt::Any, world::UInt64) @ Demo2 /Users/aviatesk/julia/JET-20230703/Demo2/src/Demo2.jl:10
││┌ infer(tt::Any, world::UInt64, interp::Core.Compiler.NativeInterpreter) @ Demo2 /Users/aviatesk/julia/JET-20230703/Demo2/src/Demo2.jl:18
│││┌ typeinf(interp::Core.Compiler.NativeInterpreter, frame::Core.Compiler.InferenceState) @ Core.Compiler ./compiler/typeinfer.jl:216
││││┌ _typeinf(interp::Core.Compiler.NativeInterpreter, frame::Core.Compiler.InferenceState) @ Core.Compiler ./compiler/typeinfer.jl:246
│││││┌ typeinf_nocycle(interp::Core.Compiler.NativeInterpreter, frame::Core.Compiler.InferenceState) @ Core.Compiler ./compiler/abstractinterpretation.jl:2969
││││││┌ typeinf_local(interp::Core.Compiler.NativeInterpreter, frame::Core.Compiler.InferenceState) @ Core.Compiler ./compiler/abstractinterpretation.jl:2822
│││││││┌ widenreturn(rt::Any, info::Core.Compiler.Best

**Caution:**

Don't do `report_package(Demo2, target_modules=(Demo2,))`. 
Use `target_defined_modules` when you use `report_package`.
(This caveat will hopefully be improved soon.)

## Tips for `report_package`

- Declare as precise method signature as possible.
- Use configurations wisely.

# Configurations

Note: `toplevel_logger=nothing` below is not necessary outside of notebook, but it produces noisy log within notebook output view.

In [9]:
report_package("JuliaHub"; toplevel_logger=nothing)

═════ 306 possible errors found ═════
┌ _parse_response_json(r::HTTP.Messages.Response, ::Type{T}) where T @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/utils.jl:127
│┌ _parse_response_json(s::AbstractString, ::Type{T}) where T @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/utils.jl:131
││┌ parse(str::AbstractString) @ JSON.Parser /Users/aviatesk/.julia/packages/JSON/93Ea8/src/Parser.jl:443
│││┌ parse(str::AbstractString; dicttype::Type{Dict{String, Any}}, inttype::Type{Int64}, allownan::Bool, null::Nothing) @ JSON.Parser /Users/aviatesk/.julia/packages/JSON/93Ea8/src/Parser.jl:450
││││┌ parse_value(pc::JSON.Parser.ParserContext{Dict{String, Any}, Int64, true, nothing}, ps::JSON.Parser.MemoryParserState) @ JSON.Parser /Users/aviatesk/.julia/packages/JSON/93Ea8/src/Parser.jl:164
│││││┌ parse_number(pc::JSON.Parser.ParserContext{Dict{String, Any}, Int64, true, nothing}, ps::JSON.Parser.MemoryParserState) @ JSON.Parser /Users/aviatesk/.julia/packages/JSON/93Ea8/sr

In [10]:
report_package("JuliaHub"; target_defined_modules=true, toplevel_logger=nothing)

═════ 107 possible errors found ═════
┌ _authenticate(server_uri::URIs.URI; force::Bool, maxcount::Integer, hook::Union{Nothing, Function, Type}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/authentication.jl:227
│ no matching method found `register_open_browser_hook(::Nothing)` (1/3 union split): PkgAuthentication.register_open_browser_hook(hook::Union{Nothing, Function, Type})
└────────────────────
┌ _authentication(server::URIs.URI; token::JuliaHub.Secret, expires::Union{Nothing, Number}, email::Union{Nothing, AbstractString}, username::Union{Nothing, AbstractString}, tokenpath::Union{Nothing, AbstractString}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/authentication.jl:340
│ no matching method found `kwcall(::NamedTuple{(:email, :expires, :tokenpath), _A} where _A<:Tuple{Union{Nothing, AbstractString}, Union{Nothing, Number}, Union{Nothing, AbstractString}}, ::Type{JuliaHub.Authentication}, ::URIs.URI, ::VersionNumber, ::Nothing, ::JuliaHub.Secret)` (1

Sometimes `report_package` produces very noisy results even with `target_defined_modules`.
In such cases, `mode=:typo` can be useful as a first step to improve your code.

In [11]:
report_package("JuliaHub"; mode=:typo, target_defined_modules=true, toplevel_logger=nothing)

═════ 4 possible errors found ═════
┌ batchimages(product::Union{Nothing, AbstractString}; auth::JuliaHub.Authentication) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/batchimages.jl:68
│┌ _batchimages_62(auth::JuliaHub.Authentication) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/batchimages.jl:286
││┌ _product_image_groups(auth::JuliaHub.Authentication) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/batchimages.jl:193
│││┌ map(f::JuliaHub.var"#67#68"{String}, A::Vector) @ Base ./abstractarray.jl:3261
││││┌ collect_similar(cont::Vector, itr::Base.Generator{_A, JuliaHub.var"#67#68"{String}} where _A) @ Base ./array.jl:711
│││││┌ _collect(c::Vector, itr::Base.Generator{_A, JuliaHub.var"#67#68"{String}} where _A, ::Base.EltypeUnknown, isz::Union{Base.HasLength, Base.HasShape}) @ Base ./array.jl:802
││││││┌ iterate(::Base.Generator{_A, JuliaHub.var"#67#68"{String}} where _A) @ Base ./generator.jl:47
│││││││┌ (::JuliaHub.var"#67#68"{String})(::Any) @ J

Let's look at the default mode output again.

There are lots of `no matching method found` errors.

Where should we start to look at?

In [12]:
result = report_package("JuliaHub"; target_defined_modules=true, toplevel_logger=nothing)

═════ 107 possible errors found ═════
┌ _authenticate(server_uri::URIs.URI; force::Bool, maxcount::Integer, hook::Union{Nothing, Function, Type}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/authentication.jl:227
│ no matching method found `register_open_browser_hook(::Nothing)` (1/3 union split): PkgAuthentication.register_open_browser_hook(hook::Union{Nothing, Function, Type})
└────────────────────
┌ _authentication(server::URIs.URI; token::JuliaHub.Secret, expires::Union{Nothing, Number}, email::Union{Nothing, AbstractString}, username::Union{Nothing, AbstractString}, tokenpath::Union{Nothing, AbstractString}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/authentication.jl:340
│ no matching method found `kwcall(::NamedTuple{(:email, :expires, :tokenpath), _A} where _A<:Tuple{Union{Nothing, AbstractString}, Union{Nothing, Number}, Union{Nothing, AbstractString}}, ::Type{JuliaHub.Authentication}, ::URIs.URI, ::VersionNumber, ::Nothing, ::JuliaHub.Secret)` (1

In [13]:
reports = JET.get_reports(result)

107-element Vector{JET.InferenceErrorReport}:
 MethodErrorReport(no matching method found `register_open_browser_hook(::Nothing)` (1/3 union split): PkgAuthentication.register_open_browser_hook(hook::Union{Nothing, Function, Type}))
 MethodErrorReport(no matching method found `kwcall(::NamedTuple{(:email, :expires, :tokenpath), _A} where _A<:Tuple{Union{Nothing, AbstractString}, Union{Nothing, Number}, Union{Nothing, AbstractString}}, ::Type{Main.var"##JETVirtualModule#374".JuliaHub.Authentication}, ::URIs.URI, ::VersionNumber, ::Nothing, ::Main.var"##JETVirtualModule#374".JuliaHub.Secret)` (1/2 union split): Core.kwcall(NamedTuple{(:email, :expires, :tokenpath)}(tuple(email::Union{Nothing, AbstractString}, expires::Union{Nothing, Number}, tokenpath::Union{Nothing, AbstractString})::Tuple{Union{Nothing, AbstractString}, Union{Nothing, Number}, Union{Nothing, AbstractString}})::NamedTuple{(:email, :expires, :tokenpath), _A} where _A<:Tuple{Union{Nothing, AbstractString}, Union{Nothing, 

In [14]:
unique(typeof.(reports))

3-element Vector{DataType}:
 JET.MethodErrorReport
 JET.UndefVarErrorReport
 JET.NonBooleanCondErrorReport

In [15]:
report = reports[end]

MethodErrorReport(no matching method found `+(::Int64, ::Nothing)` (1/2 union split): (((b::Main.var"##JETVirtualModule#374".JuliaHub.KafkaLogsBuffer)._active_range::UnitRange{Int64}).stop::Int64 Main.var"##JETVirtualModule#374".JuliaHub.:+ count::Union{Nothing, Integer}))

In [16]:
report.vst[end].linfo.def.module

Main.var"##JETVirtualModule#374".JuliaHub

In [17]:
report.t

1-element Vector{Any}:
 Tuple{typeof(+), Int64, Nothing}

In [18]:
struct MyFilter end
function JET.configured_reports(::MyFilter, reports::Vector{JET.InferenceErrorReport})
    filter(reports) do report
        # XXX this is necessary since a custom `report_config` overrides `target_defined_modules` setting.
        occursin("JuliaHub", string(last(report.vst).linfo.def.module)) || return false

        isa(report, JET.MethodErrorReport) || return false
        report.t isa DataType || return false
        any(==(Nothing), report.t.parameters) && return false
        return true
    end
end

result = report_package("JuliaHub"; 
                        report_config=MyFilter(), 
                        toplevel_logger=nothing)

═════ 10 possible errors found ═════
┌ _restput_mockable(args::Vararg{Any}; kwargs::Base.Pairs{Symbol, V, Tuple{Vararg{Symbol, N}}, NamedTuple{names, T}} where {V, N, names, T<:Tuple{Vararg{Any, N}}}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/restapi.jl:114
│ no matching method found `kwcall(::NamedTuple{names, T} where {N, names, T<:Tuple{Vararg{Any, N}}}, ::typeof(JuliaHub._restput_http_put), ::Vararg{Any})`: Core.kwcall(tuple(merge(Base.NamedTuple()::NamedTuple{(), Tuple{}}, kwargs::Base.Pairs{…})::NamedTuple{names, T} where {N, names, T<:Tuple{Vararg{Any, N}}}, JuliaHub._restput_http_put)::Tuple{NamedTuple{names, T} where {N, names, T<:Tuple{Vararg{Any, N}}}, typeof(JuliaHub._restput_http_put)}, args::Tuple...)
└────────────────────
┌ nodespec(; auth::JuliaHub.Authentication, kwargs::Base.Pairs{Symbol, V, Tuple{Vararg{Symbol, N}}, NamedTuple{names, T}} where {V, N, names, T<:Tuple{Vararg{Any, N}}}) @ JuliaHub /Users/aviatesk/julia/JET-20230703/JuliaHub/src/node.jl:

# Incoming

## MethodAnalysis.jl integration

Basic idea: Let the existing test suite to collect analysis entry points.

1. run test suite first
2. collect runtime method signatures
3. run static analysis on it

## Source-level annotation

- per-block
- per-method
- per-file

```julia
using JETAnnotations: @assume_safety

function foo(x::Base.RefValue{Union{T,Nothing}}) where T<:Number
    if x[] === nothing
        return zero(T)
    end
    return @assume_safety sin(x[])
end
```